### Problem 2: two generators, single load, single bus optimization problem.
- multiple time periods
- dual values

In [ ]:
### Problem formulation
# generator 1: "gas" (CCGT), marginal cost 70 EUR/MWh, capacity 50 MW
# generator 2: "coal" (hard coal), marginal cost 40 EUR/MWh, capacity 100 MW
# load: "DE" (Germany)
# multiple time steps

In [ ]:
import pypsa
import numpy as np
import pandas as pd
import linopy
import matplotlib.pyplot as plt

### Create PyPSA network with components of the problem

In [ ]:
n = pypsa.Network()

In [ ]:
n.add("Bus", "DE", v_nom=380)

In [ ]:
n.add(
    "Generator",
    "gas",
    bus="DE",
    p_nom_extendable=False,
    marginal_cost=70,  # €/MWh
    p_nom=50,  # MW
)
n.add(
    "Generator",
    "coal",
    bus="DE",
    p_nom_extendable=False,
    marginal_cost=40,  # €/MWh
    p_nom=100,  # MW
)

In [ ]:
n.snapshots = pd.date_range("2019-01-01", periods=5, freq="h")

In [ ]:
load_series = pd.Series([80, 90, 110, 120, 100], index=n.snapshots)

In [ ]:
load_series

In [ ]:
# add load
n.add(
    "Load",
    "Germany",
    bus="DE",
    p_set=load_series,  # MW
)

### Solve with PyPSA optimize module (use the default mathematical problem)

In [ ]:
# n.optimize.create_model()

In [ ]:
n.optimize(solver_name="glpk")

In [ ]:
n.objective

In [ ]:
n.generators_t.p

In [ ]:
def plot_dispatch(n):
    df = n.generators_t.p
    colors = {"coal": "#8B4513", "gas": "#FFA500"}
    df = df[["coal", "gas"]]
    df.plot(kind="bar", stacked=True, width=0.9, color=[colors[c] for c in df.columns])
    plt.xticks(np.arange(len(df.index)), df.index.strftime("%H:%M"))
    plt.xticks(rotation=0)
    plt.xlabel("Time")
    plt.ylabel("MW")
    plt.legend(loc="upper left")
    plt.show()


plot_dispatch(n)

### Explore & discuss duals

In [ ]:
n.model.dual

In [ ]:
# https://www.epexspot.com/en/market-data